<a href="https://colab.research.google.com/github/hwankang/Deep-Chemometrics/blob/master/18(fri)_11_16pm_unscaling_method_complete.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
import scipy.io as sio
import numpy as np

def get_xY(filename, maxx=600):
    
    #sio.whosmat(filename)

    matcontents = sio.loadmat(filename)
    keys = matcontents.keys()
    #for key in list(keys):
    #    if key[0] == '_':
    #        keys.remove(key)
            
    #keys.sort()
            
    d = {}            
    for key in keys:
        data = matcontents[key][0][0]
        if key[-1] == "Y":
            Ydata = data[5]
            d[key] = Ydata
        else:
            xdata = data[5][:,:maxx]
            d[key] = xdata
            d["axisscale"]= data[7][1][0][0][:maxx].astype(np.float)
                    
    return d

In [ ]:
filename='/content/drive/MyDrive/machine_learning/nir_shootout_2003.mat'
#dataset = get_xY(filename)
#from scipy.io import loadmat
#filename = 'Dataset/nir_shootout_2002.mat'
matcontents = sio.loadmat(filename)
keys = matcontents.keys()
list(keys)
data=matcontents['test_1'][0][0]
X_train=data[7][1][0][0][:600].astype(np.float)
#dataset = get_xY(filename)
X_train
#data1=matcontents['axisscale'][0][0]
#axisscale=data1[7][1][0][0][:600].astype(np.float)
#axisscale

In [ ]:
xdata=data[5][:,:600]
xdata

In [ ]:
keys = matcontents.keys()
list(keys)

In [ ]:
maxx=600
d = {}  
key2=[ 'calibrate_1',
 'calibrate_2',
 'test_1',
 'test_2',
 'validate_1',
 'validate_2',
 'calibrate_Y',
 'test_Y',
 'validate_Y']          
for key in key2:
    data = matcontents[key][0][0]
    if key[-1] == "Y":
        Ydata = data[5]
        d[key] = Ydata
    else:
        xdata = data[5][:,:maxx]
        d[key] = xdata
        d["axisscale"]= data[7][1][0][0][:maxx].astype(np.float)

In [ ]:
d

In [ ]:
dataset=d
import matplotlib.pyplot as plt
%matplotlib inline

#from ChemUtils import GlobalStandardScaler

#xscaler = GlobalStandardScaler()
X_train = dataset['test_1']
X_test = dataset['calibrate_2']
#X_train = xscaler.fit_transform(dataset['test_1'])
#X_test = xscaler.transform(dataset['calibrate_2'])
plt.plot(dataset['axisscale'],X_train.T)
plt.show()
#print X_train.std()
#print X_train.mean()
#print X_train.max()
#_ = plt.plot(dataset['axisscale'],X_train.T)

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv1D, Reshape#, MaxPooling1D
from keras.callbacks import ReduceLROnPlateau
from keras.layers.noise import GaussianNoise

In [ ]:
#Hyperparameters for the network
DENSE = 128
DROPOUT = 0.5
C1_K  = 8 #Number of kernels/feature extractors for first layer
C1_S  = 32 #Width of the convolutional mini networks
C2_K  = 16
C2_S  = 32

activation='relu'

input_dim = X_train.shape[1]

#The model
def make_model():
    model = Sequential()
    #Adding a bit of GaussianNoise also works as regularization
    model.add(GaussianNoise(0.05, input_shape=(input_dim,)))
    #First two is number of filter + kernel size
    model.add(Reshape((input_dim, 1)))
    #model.add(Conv1D(C1_K, (C1_S), activation=activation, border_mode='same'))
    #model.add(Conv1D(C2_K, (C2_S), border_mode='same', activation=activation))
    model.add(Conv1D(C1_K, (C1_S), activation=activation, padding='same'))
    model.add(Conv1D(C2_K, (C2_S), padding='same', activation=activation))
    model.add(Flatten())
    model.add(Dropout(DROPOUT))
    model.add(Dense(DENSE, activation=activation))
    model.add(Dense(1, activation='linear'))

    model.compile(loss='mse', optimizer=keras.optimizers.Adadelta(lr=0.01))#, metrics=['mean_absolute_error'])

    return model

In [ ]:
model = make_model()
print(model.summary())

In [ ]:
#y_train = yscaler.fit_transform(dataset['test_Y'][:,2])
#y_test = yscaler.transform(dataset['calibrate_Y'][:,2]) 
y_train = dataset['test_Y'][:,2]
y_test = dataset['calibrate_Y'][:,2]
y_test.size
X_train.shape

In [ ]:
#h = model.fit(X_train, y_train, epochs=600, batch_size=8, validation_data=(X_test, y_test), callbacks=[rdlr])
h = model.fit(X_train, y_train, epochs=600, batch_size=8, validation_data=(X_test, y_test))

In [ ]:
plt.plot(h.history['loss'], label='loss')
plt.plot(h.history['val_loss'], label='val_loss')

#plt.yscale('log')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend()
ax2 = plt.gca().twinx()
ax2.plot(h.history['lr'], color='r')
ax2.set_ylabel('lr',color='r')

_ = plt.legend()

In [ ]:
plt.scatter(y_train, model.predict(X_train))
plt.scatter(y_test, model.predict(X_test))
plt.plot([100,250],[150,250]) # Y = PredY line

In [ ]:
pred_Y_df=model.predict(X_test)
Y_df_true=y_test

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
mse= mean_squared_error(Y_df_true, pred_Y_df)
rmse=np.sqrt(mse)
r2=r2_score(Y_df_true, pred_Y_df)
print('MSE : {: 0.3f}, || RMSE : {: 0.3f}, || R2 : {: 0.3f}'.format(mse,rmse,r2))

end

In [ ]:
!git clone https://github.com/EBjerrum/Deep-Chemometrics

In [ ]:
!pip install ChemUtils

In [ ]:
filename='/content/drive/MyDrive/machine_learning/nir_shootout_2003.mat'
#dataset = get_xY(filename)
from scipy.io import loadmat
#datafile = "yourfile.mat"
datafile = filename
data = loadmat(datafile, matlab_compatible=True)
#data.head()
validate_1 = data['validate_1'].squeeze()
validate_2 = data['validate_2'].squeeze()
validate_Y = data['validate_Y'].squeeze()
test_Y = data['test_Y'].squeeze()
calibrate_Y = data['calibrate_Y'].squeeze()
test_2 = data['test_2'].squeeze()
test_1 = data['test_1'].squeeze()
calibrate_1= data['calibrate_1'].squeeze()
calibrate_2= data['calibrate_2'].squeeze()
#axisscale= data['axisscale'].squeeze()
#var2 = data['nameOfYourOtherVariable'].squeeze()
#[출처] Python을 사용하여 MATLAB 대체: 데이터를 가져오는 방법?|작성자 animepolis185135



In [ ]:
import scipy.io as sio
import scipy
import numpy as np
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML(""))

from sys import stdout
import os
import sys
import random

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.collections as collections
import seaborn as sns 
from scipy.signal import savgol_filter
from scipy.optimize import minimize_scalar
import scipy.linalg as linalg
from cycler import cycler

from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler, Binarizer, MinMaxScaler
from sklearn.cross_decomposition import PLSRegression
from sklearn.model_selection import train_test_split, cross_val_predict,cross_val_score , KFold
from sklearn.metrics import mean_squared_error, r2_score, make_scorer, explained_variance_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, cross_val_predict, cross_val_score, KFold , \
    cross_validate, StratifiedKFold

In [ ]:
#!pip install -y tensorflow keras
!git clone https://github.com/EBjerrum/Deep-Chemometrics
#!git clone https://github.com/EBjerrum

In [ ]:
#%cd Deep-Chemometrics
!pip install ChemUtils

In [ ]:
from ChemUtils import GlobalStandardScaler, EmscScaler

import tensorflow as tf
from tensorflow import keras 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv1D, Reshape#, MaxPooling1D
from tensorflow.keras.layers import GaussianNoise
from tensorflow.keras.callbacks import EarlyStopping,ReduceLROnPlateau
from livelossplot.tf_keras import PlotLossesCallback
# from livelossplot.tf_keras import PlotLossesCallback
# from livelossplot.keras import PlotLossesCallback


## Define random seeds ir order to maintain reproducible results through multiple testing phases
def set_reproducible():
    os.environ['PYTHONHASHSEED'] = '0'
    np.random.seed(12345)
    random.seed(12345)
    tf.random.set_seed(12345)
    
set_reproducible()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

#from ChemUtils import GlobalStandardScaler

#xscaler = GlobalStandardScaler()

#X_train = xscaler.fit_transform(dataset['test_1'])
#X_train = xscaler.fit_transform(test_1)
X_train=test_1
#X_test = xscaler.transform(calibrate_2)
X_test=calibrate_2
#X_test = xscaler.transform(dataset['calibrate_2'])
X_train.head()
#X_train.std()
#print X_train.mean()
#print X_train.max()
#_ = plt.plot(dataset['axisscale'],X_train.T)